In [42]:
import numpy as np
import pandas as pd
import math
import os
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error

#######딥러닝 라이브러리##########
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Reshape, GRU, RNN

tf.keras.backend.set_floatx('float64')

In [43]:
os.listdir('../energy')

['train.csv', 'sample_submission.csv', 'test.csv']

In [44]:

train=pd.read_csv('../energy/train.csv', encoding='CP949')

In [45]:
test=pd.read_csv('../energy/test.csv', encoding='CP949')
submission=pd.read_csv('../energy/sample_submission.csv', encoding='CP949')

In [46]:
train

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
0,1,2020-06-01 00,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0
1,1,2020-06-01 01,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0
2,1,2020-06-01 02,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0
3,1,2020-06-01 03,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0
4,1,2020-06-01 04,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
122395,60,2020-08-24 19,4114.368,27.8,2.3,68.0,0.0,0.7,1.0,1.0
122396,60,2020-08-24 20,3975.696,27.3,1.2,71.0,0.0,0.0,1.0,1.0
122397,60,2020-08-24 21,3572.208,27.3,1.8,71.0,0.0,0.0,1.0,1.0
122398,60,2020-08-24 22,3299.184,27.1,1.8,74.0,0.0,0.0,1.0,1.0


In [47]:
mini=train.iloc[:,2].min()
size=train.iloc[:,2].max()-train.iloc[:,2].min()
train.iloc[:,2]=(train.iloc[:,2]-mini)/size

In [74]:
input_window =168 #임의의 수
output_window = 1 #168 7일 24시간
window = 12 #window는 12시간 마다는 12시간 마다
num_features = 1 #베이스라인은 feature를 하나만 사용했습니다.
num_power = 60
end_=168
lstm_units=32
dropout=0.2
EPOCH=30
BATCH_SIZE=128

In [75]:
train_x=tf.reshape(train.iloc[:,2].values, [num_power, 24*85, num_features])
#print(f'train_x.shape:{train_x.shape}')

In [76]:
train_window_x= np.zeros(( train_x.shape[0], (train_x.shape[1]-(input_window + output_window))//window, input_window, num_features)) 
train_window_y= np.zeros(( train_x.shape[0], (train_x.shape[1]-(input_window + output_window))//window, output_window, num_features))
print(train_window_x.shape)
print(train_window_y.shape)

(60, 155, 168, 1)
(60, 155, 1, 1)


In [77]:
for example in range(train_x.shape[0]):
    
    for start in range(0, train_x.shape[1]-(input_window+output_window), window):
        end=start+input_window
        train_window_x[example, start//window, :] = train_x[example, start: end               , :]
        train_window_y[example, start//window, :] = train_x[example, end  : end+ output_window, :]


IndexError: index 155 is out of bounds for axis 1 with size 155

In [38]:
new_train_x=tf.reshape(train_window_x, [-1, input_window, num_features])
new_train_y=tf.reshape(train_window_y, [-1, output_window,num_features])
#print(f'new_train_x.shape:{new_train_x.shape}')
#print(f'new_train_y.shape:{new_train_y.shape}')

In [27]:
996/24

41.5

In [29]:
from tensorflow.keras.layers import *


In [55]:
first_input = Input(shape=(996,1))


in0=LSTM(64,return_sequences=True)(first_input)
#in1=add([first_input,in0])
in0=Conv1D(filters=64, kernel_size=3, activation='relu',padding='same')(in0)
#in1= BatchNormalization(scale=False)(in0)

in1=Conv1D(filters=64, kernel_size=2, activation='relu',padding='same')(in0)
#in1= BatchNormalization(scale=False)(in1)
#in1=Conv1D(filters=64, kernel_size=3, activation='relu',padding='same')(in0)

in1=Conv1D(filters=64, kernel_size=1, activation='relu',padding='same')(in1)
#in1= BatchNormalization(scale=False)(in1)

ad=add([in0,in1],name='extractor1')

#ad=Dropout(0.5)(ad)
#in0=LSTM(64,return_sequences=True)(ad)
#in1=add([first_input,in0])
in0=Conv1D(filters=64, kernel_size=3, activation='relu',padding='same')(ad)
#in1= BatchNormalization(scale=False)(in0)

in1=Conv1D(filters=64, kernel_size=2, activation='relu',padding='same')(in0)
#in1= BatchNormalization(scale=False)(in1)
#in1=Conv1D(filters=64, kernel_size=3, activation='relu',padding='same')(in0)

in1=Conv1D(filters=64, kernel_size=1, activation='relu',padding='same',name='extractor')(in1)
#in1= BatchNormalization(scale=False)(in1)

ad1=add([in0,in1])


#in0=Conv1D(filters=64, kernel_size=3, activation='relu',padding='same')(ad1)
#in1= BatchNormalization(scale=False)(in0)

#in1=Conv1D(filters=64, kernel_size=3, activation='relu',padding='same')(in0)
#in1= BatchNormalization(scale=False)(in1)
#in1=Conv1D(filters=64, kernel_size=3, activation='relu',padding='same')(in0)

#in1=Conv1D(filters=64, kernel_size=1, activation='relu',padding='same')(in1)
#in1= BatchNormalization(scale=False)(in1)

#in1=add([ad1,in1],name='extractor2')
#in0=Conv1D(filters=64, kernel_size=5, activation='relu',padding='same')(in1)
#in1= BatchNormalization(scale=False)(in0)

#in1=Conv1D(filters=64, kernel_size=3, activation='relu',padding='same')(in0)
#in1= BatchNormalization(scale=False)(in1)
#in1=Conv1D(filters=64, kernel_size=3, activation='relu',padding='same')(in0)

#in1=Conv1D(filters=64, kernel_size=1, activation='relu',padding='same')(in1)
#in1= BatchNormalization(scale=False)(in1)

#in1=add([in0,in1])

#attn_layer = AttentionLayer(name='attention_layer')
#attn_out, attn_states = attn_layer([in1,in1])

    # Concat attention input and LSTM output, in original code it was decoder LSTM
#concat_out = Concatenate(axis=-1, name='concat_layer')([in1, attn_out])
#in1=MaxPooling1D(pool_size=2, name='extractor')(in1)
in1=Flatten()(ad1)
in1=Dropout(0.5)(in1)
#in1=Dense(100, activation='relu')(in1)
out=Dense(24,'relu')(in1)
model=tf.keras.Model(inputs=[first_input],outputs=[out])
model.compile(optimizer='adam', loss='mse')

model.summary()

checkpoint_path = "saved_weights/bike_reslstmcnn_multi.hdf5"
cb = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_best_only=True,
                                        monitor='val_loss',mode='min',
                                                 verbose=1)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 996, 1)]     0                                            
__________________________________________________________________________________________________
lstm_4 (LSTM)                   (None, 996, 64)      16896       input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 996, 64)      12352       lstm_4[0][0]                     
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 996, 64)      8256        conv1d_5[0][0]                   
____________________________________________________________________________________________

In [53]:
model.compile(optimizer='rmsprop', loss='mae', metrics=['mae'])
# 에포크가 끝날 때마다 점(.)을 출력해 훈련 진행 과정을 표시합니다
class PrintDot(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 10 == 0: print('')
        print('.', end='')

#가장 좋은 성능을 낸 val_loss가 적은 model만 남겨 놓았습니다.
save_best_only=tf.keras.callbacks.ModelCheckpoint(filepath="lstm_model.h5", monitor='val_loss', save_best_only=True)


early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

#검증 손실이 10epoch 동안 좋아지지 않으면 학습률을 0.1 배로 재구성하는 명령어입니다.
reduceLR = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)


In [65]:
model.fit(new_train_x, new_train_y, epochs=EPOCH, batch_size=BATCH_SIZE, validation_split = 0.2, verbose=1,
          callbacks=[early_stop, save_best_only , reduceLR])

model.summary()

Train on 4080 samples, validate on 1020 samples
Epoch 1/30
4080/4080 [==============================] - 46s 11ms/sample - loss: 5.0687e-04 - val_loss: 2.8190e-04
Epoch 2/30
4080/4080 [==============================] - 47s 11ms/sample - loss: 4.6384e-04 - val_loss: 2.9547e-04
Epoch 3/30
4080/4080 [==============================] - 46s 11ms/sample - loss: 4.3178e-04 - val_loss: 2.6654e-04
Epoch 4/30
4080/4080 [==============================] - 47s 11ms/sample - loss: 4.4042e-04 - val_loss: 2.4161e-04
Epoch 5/30
4080/4080 [==============================] - 46s 11ms/sample - loss: 4.3925e-04 - val_loss: 2.3228e-04
Epoch 6/30
4080/4080 [==============================] - 47s 12ms/sample - loss: 4.0177e-04 - val_loss: 3.0696e-04
Epoch 10/30
4080/4080 [==============================] - 46s 11ms/sample - loss: 4.1385e-04 - val_loss: 2.8849e-04
Epoch 11/30
4080/4080 [==============================] - 47s 12ms/sample - loss: 4.7567e-04 - val_loss: 2.6935e-04
Epoch 12/30
4080/4080 [===============

In [66]:
model.load_weights('lstm_model.h5')

In [67]:
prediction=np.zeros((num_power, end_, num_features))
new_test_x=train_x

for i in range(end_//output_window):
    start_=i*output_window
    next_=model.predict(new_test_x[ : , -input_window:, :])
    next_=next_.reshape(-1,24,1)
    new_test_x = tf.concat([new_test_x, next_], axis=1)
    print(new_test_x.shape)
    prediction[:, start_: start_ + output_window, :]= next_
prediction =prediction *size + mini

(60, 2064, 1)
(60, 2088, 1)
(60, 2112, 1)
(60, 2136, 1)
(60, 2160, 1)
(60, 2184, 1)
(60, 2208, 1)


In [63]:
submission['answer']=prediction.reshape([-1,1])
submission

,num_date_time,answer
0,1 2020-08-25 00,8389.011114
1,1 2020-08-25 01,8400.459102
2,1 2020-08-25 02,8521.641314
3,1 2020-08-25 03,8746.405459
4,1 2020-08-25 04,8748.448954
...,...,...
10075,60 2020-08-31 19,4188.186717
10076,60 2020-08-31 20,4094.915336
10077,60 2020-08-31 21,3782.975027
10078,60 2020-08-31 22,3536.422318


In [68]:
submission['answer']=prediction.reshape([-1,1])
submission

,num_date_time,answer
0,1 2020-08-25 00,8425.001242
1,1 2020-08-25 01,8371.929490
2,1 2020-08-25 02,8369.780149
3,1 2020-08-25 03,8321.061479
4,1 2020-08-25 04,8317.540518
...,...,...
10075,60 2020-08-31 19,4053.047840
10076,60 2020-08-31 20,3993.458084
10077,60 2020-08-31 21,3706.933413
10078,60 2020-08-31 22,3391.136883


In [69]:
submission.to_csv('baseline_submission1.csv', index=False)